Drawn from https://www.statsmodels.org/dev/examples/notebooks/generated/glm_formula.html.

In [2]:
# Summon libraries.
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import matplotlib
import scipy
import numpy as np
import sys as sys
import os

Call data.

In [3]:
# Call data. Uses PANDAS.
drugs = pd.read_csv('../../data/Exercise1.2Data.csv')
drugs.head()

,drug,age,gender,EWL
0,A,49,F,14.2
1,A,54,M,25.4
2,A,37,F,14.1
3,A,43,F,20.0
4,A,57,M,11.7


Normality.
https://machinelearningmastery.com/a-gentle-introduction-to-normality-tests-in-python/

In [ ]:
# Sets the y-value to the y-value[name] for normality tests.
y_val = drugs["EWL"]

from matplotlib import pyplot
# Summon histogram. Uses matplotlib.
pyplot.hist(y_val,5)
pyplot.show()

# Uses scipy.stats for an Anderson-Darling test.
from scipy.stats import anderson
# Assign result from an Anderson-Darling test using the y-value.
result = anderson(y_val)
# Dump out result's test statistic.
print(f'Anderson-Darling test statistic: {result.statistic:.3f}')

# Uses scipy.stats for a Shapiro-Wilks test.
from scipy.stats import shapiro
# Assign results from a Shapiro-Wilks test using the y-value.
statistic, p_value = shapiro(y_val)
print(f"""Shapiro-Wilks test statistic: {statistic:.3f} 
Shapiro-Wilks p-value: {p_value:.3f}""")

Develop models.

In [ ]:
# Develop model.
formula = 'EWL ~ age + C(drug,Treatment("A")) + C(gender,Treatment("M"))'
fitted = smf.glm(formula=formula, data=drugs, family=sm.families.Gaussian()).fit()
print(fitted.summary())
fitloglike = (fitted.llf)
print(f'Sigma of fitted model is {np.sqrt(fitted.scale)}.')

In [ ]:
# Null model.
formula = 'EWL ~ 1'
null = smf.glm(formula=formula, data=drugs, family=sm.families.Gaussian()).fit()
nullloglike = (null.llf)

Log likelihood. Found the names through dir() https://stackoverflow.com/questions/2675028/list-attributes-of-an-object.

In [ ]:
# Uses null and fitted log likelihoods to perform the deviance test.
deviance= -2 * (nullloglike-(fitloglike))
print(f"Deviance statistic is {deviance}.")
# Chi2.cdf is from scipy.stats.
from scipy.stats import chi2
pvalue = 1 - chi2.cdf(deviance,3)
print(f"p-value is {pvalue}.")

Prediction. https://www.statology.org/statsmodels-predict/

In [ ]:
# Prediction.
predict_val = pd.DataFrame(
    {"age" : 35, "gender" : 'M', "drug" : "A"}, index=[0])
predict_val = sm.add_constant(predict_val)
fitted.predict(predict_val)